### 분류 모델 개발

In [ ]:
# 이미지 로드함수 불러오기 

import os 
import re 
import cv2 
import numpy as np 


def image_load(path): 
    file_list = os.listdir(path) 
    file_name=[] 

    for  i  in  file_list: 
        a = int( re.sub('[^0-9]','', i) )   # i 가 숫자가 아니면 null 로 변경해라 ~ 
        file_name.append(a) 

    file_name.sort() # 정렬작업  

    file_res=[] 

    for  j  in  file_name: 
        file_res.append('%s/%d.jpg' %(path,j) ) 
    image=[] 

    for  k  in  file_res: 
        img = cv2.imread(k) 
        image.append(img) 

    return  np.array(image) 


# numpy로 바꾸는 함수 

import csv 

def  label_load(path): 
    file = open(path) 
    labeldata = csv.reader(file) 
    labellist = [] 
    for  i  in  labeldata: 
        labellist.append(int(i[0])) 

    return   np.array(labellist) 

In [ ]:
# 1. 데이터를 불러옵니다. 

 

# 이미지 데이터와 라벨위치  

train_image = '/content/drive/MyDrive/dogcat/train_resize/' 
test_image = '/content/drive/MyDrive/dogcat/test_resize/' 
train_label = '/content/drive/MyDrive/dogcat/train_label.csv' 
test_label = '/content/drive/MyDrive/dogcat/test_label.csv' 

# 데이터 로드  

x_train = image_load(train_image) 
y_train = label_load(train_label) 
x_test = image_load(test_image) 
y_test = label_load(test_label)      

print(x_train.shape)       # (1595,32,32,3) 
print(x_test.shape)         # (100, 32, 32, 3) 
print(y_train.shape)       # (1595,) 
print(y_test.shape)          # (100,) 

In [ ]:
# 1-2. 필요한 패키지 가져오는 코드  


import tensorflow as tf   # 텐써 플로우 2.0  
from tensorflow.keras.datasets.mnist import load_data  # 텐써플로우에 내장되어있는 mnist 데이터를 가져온다. 
from tensorflow.keras.models import Sequential  # 모델을 구성하기 위한 모듈 
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Conv2D, MaxPooling2D ,Flatten# 완전 연결계층을 구성하기 위한 모듈 
from tensorflow.keras.utils import to_categorical # one encoding 하는 모듈 
from tensorflow.keras.optimizers import Adam 

tf.random.set_seed(777) 

In [ ]:
# 2. 정규화 진행   

x_train = x_train.reshape(-1, 32, 32, 3) / 255 
x_test = x_test.reshape(-1, 32,32, 3) / 255 

# 3. 정답 데이터를 준비한다.  

# 하나의 숫자를 one hot encoding 한다. (예:  4 ---> 0 0 0 0 1 0 0 0 0 0 )  

y_train = to_categorical(y_train)  # 훈련 데이터의 라벨(정답)을 원핫 인코딩 
y_test = to_categorical(y_test)    # 테스트 데이터의 라벨(정답)을 원핫 인코딩  

print(x_train.shape) 
print(x_test.shape) 
print(y_train.shape) 
print(y_test.shape) 

In [ ]:
from tensorflow.keras.applications import * 
resnet50 = ResNet50( weights = None, input_shape = None, include_top = True ) 

In [ ]:
# 4. 모델을 구성합니다. 3층 신경망으로 구성 

model = Sequential() 

model.add(resnet50) 

model.add( Flatten() ) 
model.add(Dense(100, activation = 'relu', input_shape = (32*32, )))  # 1층 
model.add(BatchNormalization()) 
model.add(Dropout(0.3)) 
model.add(Dense(100, activation = 'relu') ) # 2층 은닉층  
model.add(BatchNormalization()) 
model.add(Dropout(0.3)) 
model.add(Dense(2, activation = 'softmax'))  # 3층 출력층  

In [ ]:
# 5. 모델을 설정합니다. ( 경사하강법, 오차함수를 정의해줍니다. ) 


model.compile(optimizer=Adam(1e-4),                         # 정확도가 들쭉날쭉하면 러닝레이트를 해주자 
                     loss = 'binary_crossentropy',  
                     metrics=['acc'])  # 학습과정에서 정확도를 보려고  

#6. 모델을 훈련시킵니다.  
from tensorflow.keras.callbacks import EarlyStopping 


# 콜백을 정의합니다. 
callbacks = [EarlyStopping(monitor = 'val_acc', patience = 20, verbose = 1)] 

# callbacks 인자를 통해 정의한 콜백을 전달합니다. 

history= model.fit(x_train, y_train, 
         batch_size = 100, 
         validation_data = (x_test, y_test), 
         epochs = 200 , 
         callbacks = callbacks) 

In [ ]:
# 모델 저장 

model.save('/content/drive/MyDrive/dogcat/dogcat_vgg16.h5') 

In [ ]:
# 7.모델을 평가합니다. (오차, 정확도가 출력됩니다.) 

model.evaluate(x_test, y_test) 

In [ ]:
train_acc_list=history.history['acc'] 
# train_acc_list 

test_acc_list=history.history['val_acc'] 
# test_acc_list 

In [ ]:
import  matplotlib.pyplot  as  plt 


x = np.arange( len(train_acc_list) ) 
plt.plot( x, train_acc_list, label='train acc') 
plt.plot( x, test_acc_list, label='test acc',  linestyle='--') 
plt.ylim(0, 1) 
plt.legend(loc='lower right') 
plt.xlabel('epochs') 
plt.ylabel('accuracy') 
plt.show() 

In [ ]:
# 사진 한장씩 넣어보기 코드 

# 1. 모델을 불러옵니다. 

from tensorflow.keras.models import load_model 

new_model = load_model('/content/drive/MyDrive/dogcat/dogcat_vgg16.h5') 

for i in range(1,101): 

    img_path = '/content/drive/MyDrive/dogcat/test/'+str(i)+'.jpg' 

    img = cv2.imread(img_path) 

    #img = cv2.bitwise_not(img) 

    # print(img.shape) 

    import tensorflow.compat.v1 as tf 

    tf.disable_v2_behavior() 

    resized_images=tf.image.resize_images(img, (32, 32)) 

    # print(resized_images.shape) 

    import numpy as np 


    x2 =tf.reshape(resized_images, [-1,32,32,3]) 

    # print(x2.shape) 

    from tensorflow.keras.models import load_model 

    new_model = load_model('/content/drive/MyDrive/dogcat/dogcat_vgg16.h5') 
    results = new_model.predict(x2,steps=1) 

    print(np.argmax(results))  # 1 이 강아지, 0 이 고양이 